# Step 7 : Alert Generation & Evaluation
Carta et al. (2021) — Étape 7 du pipeline

Détecter les hot events en mesurant la proportion de tweets assignés :$$R(d) = \frac{\text{nb tweets assignés le jour } d}{\text{nb total tweets le jour } d}$$Si $R(d) > \theta$, une alerte est générée.

### Libraries

In [28]:
import pandas as pd
import plotly.graph_objects as go
import sys
import os

sys.path.append(os.path.abspath(os.path.join('..')))
from src.alert_generation import *

### Data loading

In [29]:
# Tweets and Final signatures
tweets_ready = pd.read_csv('../data/for_models/tweets_features.csv')

# final_tweets_assigned = pd.read_csv('../data/for_models/tweets_assigned_SVB.csv')
final_tweets_assigned = pd.read_csv('../data/for_models/tweets_assigned_AI.csv')

tweets_ready['date'] = pd.to_datetime(tweets_ready['date'])
final_tweets_assigned['date'] = pd.to_datetime(final_tweets_assigned['date'])

# S&P500 prices
sp500_prices = pd.read_csv('../data/processed/sp500_2023.csv', skiprows=[1, 2])
sp500_prices.columns = sp500_prices.columns.str.lower()
sp500_prices = sp500_prices.rename(columns={'price': 'date'})
sp500_prices['date'] = pd.to_datetime(sp500_prices['date'])
sp500_prices = sp500_prices.sort_values('date')

### Contruction of the ground truth

- $\delta_d = |\text{closed}_{d+7} - \text{closed}_d| / \text{closed}_d$
- Event day if $\delta_d > 2\%$

In [30]:
# Application sur vos données S&P 500
gt_intervals = build_ground_truth_events(sp500_prices)
print(f"Nombre d'événements réels détectés dans le S&P 500 : {len(gt_intervals)}")

Nombre d'événements réels détectés dans le S&P 500 : 24


In [31]:
# 2. Lancez la visualisation
plot_sp500_ground_truth_simplified(sp500_prices, gt_intervals)

### Alert Generation

In [32]:
# Alert generation for SVB
# We look 1 month around the day d
START_OBS = "2023-09-01"
END_OBS = "2023-09-21"

alert_results = generate_alerts_for_period(
    all_tweets=tweets_ready,            # tweets sur la période
    assigned_tweets=final_tweets_assigned, # tweets mapés par clusters sur la période
    start_date=START_OBS,
    end_date=END_OBS,
    alert_threshold_ratio=0.20          # Seuil d'alerte (20% de discussion liée aux news)
)

print(alert_results[alert_results['is_alert']].sort_values('date'))

         date  total  assigned     ratio  is_alert
1  2023-09-04      3       2.0  0.666667      True
2  2023-09-05     10       9.0  0.900000      True


Baseline (Le bruit de fond) : En février, vous devriez avoir un ratio bas (ex: 5-8%). Les gens parlent de finance de façon éparse.

The Spike (L'alerte) : Autour du 10-13 mars, votre ratio devrait bondir au-dessus de 20-30%. C'est là que le système génère l'alerte "Hot Event".

The Cooling (Le refroidissement) : En avril, vous verrez si le sujet SVB s'essouffle ou si de nouvelles alertes apparaissent suite aux décisions de la FED.

### Evaluation : Recall, Precision and F-Score

In [34]:
# CAS 1 : CRISE SVB (Février - Avril 2023) ---
metrics_svb = evaluate_pipeline_for_period(
    alert_results, 
    gt_intervals, 
    "2023-09-01", "2023-09-21"
)

print("\nPERFORMANCE")
print(metrics_svb)

# PERFORMANCE
# {'Période': '2023-03-01 au 2023-03-21', 
# 'Événements réels (GT)': 2, 'Alertes générées': 10, 
# 'Hits (Alertes justes)': 6, 
# 'Precision': 0.6, 'Recall': 0.5, 'F-Score': 0.5455}

# PERFORMANCE
# {'Période': '2023-09-01 au 2023-09-21', 
# 'Événements réels (GT)': 1, 
# 'Alertes générées': 2, 
# 'Hits (Alertes justes)': 0, 
# 'Precision': 0.0, 'Recall': 0.0, 'F-Score': 0}


PERFORMANCE
{'Période': '2023-09-01 au 2023-09-21', 'Événements réels (GT)': 1, 'Alertes générées': 2, 'Hits (Alertes justes)': 0, 'Precision': 0.0, 'Recall': 0.0, 'F-Score': 0}


In [35]:
plot_social_heat_curve(alert_results, gt_intervals, 0.20)

Le Trade-off (Fig 14) : Si vous baissez votre alert_threshold_ratio, votre Recall va monter (vous raterez moins de crises) mais votre Precision va baisser (vous aurez plus de fausses alertes comme les rapports de résultats trimestriels mentionnés par les auteurs).

L'Importance du Recall : Comme le disent les auteurs, pour un trader, il vaut mieux une fausse alerte qu'une crise non prédite.

La Latence : Ne soyez pas surpris si l'alerte arrive quelques jours après le début de la news. Le papier explique que l'impact économique met parfois du temps à se manifester dans la discussion sociale.